In [968]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import warnings
import math
from scipy import stats
from collections import Counter

warnings.filterwarnings("ignore")

### weight of evidence ------

from https://stackoverflow.com/questions/50352210/woe-and-iv-table-in-python

In [969]:
def calc_iv(df, feature, target, pr=0):
    lst = []

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature, val, df[df[feature] == val].count()[feature], df[(df[feature] == val) & (df[target] == 1)].count()[feature]])
 
    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'Pop', 'Bad'])
    data['Good'] = data['Pop'] - data['Bad']
    
    data['Badx'] = data['Bad'] + 0.01
    data['Goodx'] = data['Good'] + 0.01
    
    data['Pop'] = data['Good'] + data['Bad']
    data['Popx'] = data['Goodx'] + data['Badx']
    
    data['Dist Good'] = round((data['Pop'] - data['Bad']) / (data['Pop'].sum() - data['Bad'].sum()), 4)
    data['Dist Bad'] = round(data['Bad'] / data['Bad'].sum(), 4)
    
    data['Dist Goodx'] = (data['Popx'] - data['Badx']) / (data['Popx'].sum() - data['Badx'].sum())
    data['Dist Badx'] = data['Badx'] / data['Badx'].sum()
    
    data['Dist Pop'] = round(data['Pop'] / data['Pop'].sum(), 4)
    data['Bad Rate'] = round(data['Bad'] / data['Pop'], 4)
    data['grp_score'] = round((data['Dist Good']/(data['Dist Good'] + data['Dist Bad']))*10, 2)
    
    data['WoE'] = np.log(data['Dist Goodx'] / data['Dist Badx'])
    data['IV'] = data['WoE'] * (data['Dist Goodx'] - data['Dist Badx'])
    
    data['Efficiency'] =  abs(data['Dist Good'] - data['Dist Bad'])/2  
    
    data = data.sort_values(by=['Variable', 'Value'], ascending=True)
    data = data.reset_index()
    mydf = data[[
       'Value',
       'Good', 'Bad',
       'Dist Good','Dist Bad',
       'WoE',
       'IV'
    ]]

    #mydf=pd.DataFrame(data=d)
   
    if pr == 1:
        print(mydf)
        
    total_iv = data['IV'].sum()
    print(total_iv)

    #return data['IV'].values[0]
    #return mydf.values
   

In [970]:
iris = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
iris.columns = ['sepal_l', 'sepal_w', 'petal_l', 'petal_w', 'type']

df = pd.get_dummies(iris.type)['Iris-setosa'].to_frame()
iris = iris.merge(df, left_index=True, right_index=True)

iris = iris.drop(['type','sepal_w','petal_l','petal_w'], axis=1)

In [971]:
iris = iris.rename(columns = {'Iris-setosa': 'target'})

In [972]:
print(iris.head())

   sepal_l  target
0      5.1       1
1      4.9       1
2      4.7       1
3      4.6       1
4      5.0       1


In [1072]:
t = pd.DataFrame(np.random.normal(1, 1, 1000))
t['target'] = 0
u = pd.DataFrame(np.random.normal(3, 1, 1000))
u['target'] = 1
data = pd.concat([t, u])
data.columns = ['x', 'target']
data.head()

,x,target
0,2.166238,0
1,0.524059,0
2,2.015551,0
3,0.746162,0
4,0.821745,0


In [1088]:
WOE_Binning(data, 'target', 'x', p_threshold=0.5)

             Value  Good  Bad  Dist Good  Dist Bad       WoE        IV
0    [-inf, 0.334)   272    8      0.272     0.008  3.525148  0.930434
1   [0.334, 0.546)    74    6      0.074     0.006  2.510775  0.170695
2   [0.546, 0.594)    19    1      0.019     0.001  2.935015  0.052819
3   [0.594, 0.683)    37    3      0.037     0.003  2.509248  0.085296
4    [0.683, 0.77)    37    3      0.037     0.003  2.509248  0.085296
5    [0.77, 0.895)    53    7      0.053     0.007  2.023143  0.093044
6   [0.895, 1.256)   163   37      0.163     0.037  1.482623  0.186769
7   [1.256, 1.291)    17    3      0.017     0.003  1.731861  0.024241
8   [1.291, 1.724)   153   87      0.153     0.087  0.564480  0.037248
9   [1.724, 1.889)    43   37      0.043     0.037  0.150245  0.000901
10  [1.889, 2.127)    44   76      0.044     0.076 -0.546448  0.017482
11   [2.127, 2.27)    21   59      0.021     0.059 -1.032708  0.039234
12   [2.27, 2.381)    11   49      0.011     0.049 -1.493220  0.056730
13  [2

In [1058]:
data.head()

,x,target,cat,tran,mybin
0,1.711,0,"(1.711, 1.737]",1.711,"[1.626, 1.835)"
1,0.936,0,"(0.936, 0.965]",0.936,"[0.84, 1.087)"
2,0.870,0,"(0.87, 0.903]",0.870,"[0.84, 1.087)"
3,1.087,0,"(1.087, 1.125]",1.087,"[1.087, 1.418)"
4,0.664,0,"(0.664, 0.694]",0.664,"[0.314, 0.84)"


based loosely on https://github.com/jstephenj14/Monotonic-WOE-Binning-Algorithm/blob/master/Monotonic%20WOE%20Binning.py

In [1069]:
def WOE_Binning(data, Y, attr, sign = True, p_threshold=0.05):
    #defaults_threshold = 1
    #column = dataset.columns[dataset.columns != Y][0]
    
    #df = data.copy()
    data['cat'] = pd.qcut(data[attr], 100)
    data[attr] = data['cat'].apply(lambda x : x.left) 
    
    column = data[attr]
    
    summary = data.groupby([column]).agg({Y:{"means":"mean","nsamples":"size","std_dev":"std"}})
    summary.columns = summary.columns.droplevel(level=0)
    summary = summary[["means","nsamples","std_dev"]]
    summary = summary.reset_index()
    summary["del_flag"] = 0
    summary["std_dev"] = summary["std_dev"].fillna(0)
    summary = summary.sort_values([attr],ascending = sign)
    summary.reset_index(inplace = True, drop = True) 
    
    i = 0
    while True:
        if i > len(summary)-2:
            break
        else:
            i = i + 1;
            x = i - 1;
           
            n_x = summary.iloc[x].nsamples
            n_i = summary.iloc[i].nsamples
            
            n = n_x + n_i
            mean_x = summary.iloc[x].means
            mean_i = summary.iloc[i].means
            
            var_x = (summary.iloc[x].std_dev)**2
            var_i = (summary.iloc[i].std_dev)**2
            
            #std deviation
            s = np.sqrt((var_x/n_x + var_i/n_i)/2)
            p = 0
            if s != 0:
                t = (mean_x - mean_i)/s
                df = 2*(n_x + n_i) - 2
                p = 1 - stats.t.cdf(t,df=df)
            if (p >= p_threshold) or (s == 0):
                #absorb data
                summary.loc[i, attr] = summary.loc[x, attr]
                summary.loc[i, 'nsamples'] = n_x + n_i
                summary.loc[i, 'means'] = (n_x*mean_x + n_i*mean_i)/n
                summary.loc[i, 'std_dev'] = np.sqrt(((n_x*var_x)+(n_i*var_i))/n)
                summary.loc[x, 'del_flag'] = 1        
            continue
            
    summary = summary[summary['del_flag'] == 0]
    summary = summary.reset_index()
    l = [-math.inf]
    for index, row in summary.iterrows():
        if index > 0:
            l.append(row[attr])
    l.append(math.inf)    
    #print(l)
    data['mybin'] = pd.cut(x=data[attr], bins=l, right=False)
    calc_iv(data, 'mybin', 'target', pr=1)
 

In [1080]:
WOE_Binning(iris, 'target', 'sepal_l', p_threshold=0.3)

ValueError: Bin edges must be unique: array([4.3  , 4.4  , 4.4  , 4.547, 4.6  , 4.6  , 4.694, 4.743, 4.8  ,
       4.8  , 4.8  , 4.9  , 4.9  , 4.9  , 4.9  , 5.   , 5.   , 5.   ,
       5.   , 5.   , 5.   , 5.029, 5.1  , 5.1  , 5.1  , 5.1  , 5.1  ,
       5.123, 5.2  , 5.2  , 5.27 , 5.4  , 5.4  , 5.4  , 5.4  , 5.5  ,
       5.5  , 5.5  , 5.5  , 5.511, 5.6  , 5.6  , 5.6  , 5.607, 5.7  ,
       5.7  , 5.7  , 5.7  , 5.7  , 5.8  , 5.8  , 5.8  , 5.8  , 5.8  ,
       5.9  , 5.9  , 6.   , 6.   , 6.   , 6.   , 6.1  , 6.1  , 6.1  ,
       6.1  , 6.2  , 6.2  , 6.234, 6.3  , 6.3  , 6.3  , 6.3  , 6.3  ,
       6.328, 6.4  , 6.4  , 6.4  , 6.4  , 6.473, 6.5  , 6.5  , 6.52 ,
       6.6  , 6.7  , 6.7  , 6.7  , 6.7  , 6.7  , 6.763, 6.8  , 6.861,
       6.9  , 6.9  , 7.008, 7.157, 7.2  , 7.255, 7.408, 7.653, 7.7  ,
       7.7  , 7.9  ]).
You can drop duplicate edges by setting the 'duplicates' kwarg

got this from github...

In [1084]:
def chimerge(data, attr, label, max_intervals):
    distinct_vals = sorted(set(data[attr])) # Sort the distinct values
    labels = sorted(set(data[label]))       # Get all possible labels
    empty_count = {l: 0 for l in labels}    # A helper function for padding the Counter()
    intervals = [[distinct_vals[i], distinct_vals[i]] for i in range(len(distinct_vals))] # Initialize the intervals for each attribute
    while len(intervals) > max_intervals:   # While loop  
        chi = []
        for i in range(len(intervals)-1):
            # Calculate the Chi2 value
            obs0 = data[data[attr].between(intervals[i][0], intervals[i][1])]
            obs1 = data[data[attr].between(intervals[i+1][0], intervals[i+1][1])]
            total = len(obs0) + len(obs1)
            count_0 = np.array([v for i, v in {**empty_count, **Counter(obs0[label])}.items()])
            count_1 = np.array([v for i, v in {**empty_count, **Counter(obs1[label])}.items()])
            count_total = count_0 + count_1
            expected_0 = count_total*sum(count_0)/total
            expected_1 = count_total*sum(count_1)/total
            chi_ = (count_0 - expected_0)**2/expected_0 + (count_1 - expected_1)**2/expected_1
            chi_ = np.nan_to_num(chi_) # Deal with the zero counts
            chi.append(sum(chi_))      # Finally do the summation for Chi2
        min_chi = min(chi)             # Find the minimal Chi2 for current iteration
        for i, v in enumerate(chi): 
            if v == min_chi:
                min_chi_index = i      # Find the index of the interval to be merged
                break
        new_intervals = []             # Prepare for the merged new data array
        skip = False
        done = False
        for i in range(len(intervals)):
            if skip:
                skip = False
                continue
            if i == min_chi_index and not done: # Merge the intervals
                t = intervals[i] + intervals[i+1]
                new_intervals.append([min(t), max(t)])
                skip = True
                done = True
            else:
                new_intervals.append(intervals[i])
        intervals = new_intervals
        
   
    l = [-math.inf]
    for i in intervals:
        l.append(i[1])
    l.append(math.inf)
    print(l)   
    data['mybin'] = pd.cut(x=data[attr], bins=l, right=False)
    calc_iv(data, 'mybin', 'target', pr=1)
    #return l

In [1085]:
chimerge(data=data, attr='x', label='target', max_intervals=10)

[-inf, 0.0614, 0.242, 0.299, 1.002, 1.333, 1.553, 1.955, 2.341, 3.021, 3.823, inf]
              Value  Good  Bad  Dist Good  Dist Bad       WoE        IV
0    [-inf, 0.0614)   180    0      0.180     0.000  9.798183  1.763479
1   [0.0614, 0.242)    57    3      0.057     0.003  2.941287  0.158812
2    [0.242, 0.299)    19    1      0.019     0.001  2.935015  0.052824
3    [0.299, 1.002)   273   27      0.273     0.027  2.313301  0.569010
4    [1.002, 1.333)   143   37      0.143     0.037  1.351726  0.143267
5    [1.333, 1.553)    87   33      0.087     0.033  0.969213  0.052332
6    [1.553, 1.955)   109   91      0.109     0.091  0.180470  0.003248
7    [1.955, 2.341)    65  135      0.065     0.135 -0.730808  0.051151
8    [2.341, 3.021)    55  325      0.055     0.325 -1.776341  0.479559
9    [3.021, 3.823)    12  328      0.012     0.328 -3.307304  1.044993
10     [3.823, inf)     0   20      0.000     0.020 -7.601402  0.152011
4.470686741599382


testing ....

In [715]:
## Import the packages
import numpy as np
from scipy import stats

In [716]:
## Define 2 random distributions
#Sample Size
N = 10
#Gaussian distributed data with mean = 2 and var = 1
a = np.random.randn(N) + 2
#Gaussian distributed data with with mean = 0 and var = 1
b = np.random.randn(N)

In [726]:
## Calculate the Standard Deviation
#Calculate the variance to get the standard deviation

#For unbiased max likelihood estimate we have to divide the var by N-1, and therefore the parameter ddof = 1
var_a = a.var(ddof=1)
var_b = b.var(ddof=1)

In [728]:
print(var_a, var_b)

0.36602433831146336 0.7540508106223665


In [731]:
#std deviation
s = np.sqrt((var_a + var_b)/2)
#s

## Calculate the t-statistics
t = (a.mean() - b.mean())/(s*np.sqrt(2/N))

## Compare with the critical t-value
#Degrees of freedom
df = 2*N - 2

#p-value after comparison with the t 
p = 1 - stats.t.cdf(t,df=df)

print("t = " + str(t))
print("p = " + str(2*p))
### You can see that after comparing the t statistic with the critical t value (computed internally) we get a good p value of 0.0005 and thus we reject the null hypothesis and thus it proves that the mean of the two distributions are different and statistically significant.

## Cross Checking with the internal scipy function
t2, p2 = stats.ttest_ind(a,b)
print("t = " + str(t2))
print("p = " + str(p2))

t = 4.20921246738193
p = 0.0005273169029311742
t = 4.20921246738193
p = 0.0005273169029312183
